# Homework

In [131]:
import pandas as pd
import numpy as np

In [132]:
df_full = pd.read_csv('bank-full.csv', sep=';')
df_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [133]:
features = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
df = df_full[features].copy()
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

# Question 1

What is the most frequent observation (mode) for the column education?

* unknown
* primary
* secondary
* tertiary

In [134]:
df.education.mode()

0    secondary
Name: education, dtype: object

In [135]:
df.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

> A/ ```secondary```

# Question 2

What are the two features that have the biggest correlation?

* age and balance
* day and campaign
* day and pdays
* pdays and previous


In [136]:
numerical = list(df.dtypes[df.dtypes != 'object'].keys())
numerical

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [137]:
correlations = { k: [] for k in numerical}

for col in numerical:
    for col2 in numerical:
        correlations[col].append(df[col].corr(df[col2])) 

correlation_matrix = pd.DataFrame(correlations)
correlation_matrix.index = numerical

correlation_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


A/ ```pdays and previous```

## Target encoding

* Now we want to encode the y variable.
* Let's replace the values yes/no with 1/0.


In [138]:
df['y'] = (df['y'] == 'yes').astype(int)

In [139]:
df.y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64

## Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value y is not in your dataframe.


In [140]:
from sklearn.model_selection import train_test_split

In [149]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
y_full_train = df_full_train.y
y_test = df_test.y
y_train = df_train.y
y_val = df_val.y

df_full_train = df_full_train.drop(columns=['y'])
df_test = df_test.drop(columns= ['y'])
df_train = df_train.drop(columns=['y'])
df_val = df_val.drop(columns=['y'])


# Question 3

* Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?
* contact
* education
* housing
* poutcome


In [151]:
from sklearn.metrics import mutual_info_score

In [173]:
categorical = list(df_train.dtypes[df_train.dtypes == 'object'].keys())
categorical

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [177]:
mi = df_train[categorical].apply(lambda s: mutual_info_score(s, y_train))
mi.sort_values(ascending=False)

poutcome     0.029533
month        0.025090
contact      0.013356
housing      0.010343
job          0.007316
education    0.002697
marital      0.002050
dtype: float64

> A/ ```poutcome```

# Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    * model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?
* 0.6
* 0.7
* 0.8
* 0.9

In [181]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [213]:
dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [223]:
dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.fit_transform(dicts)
y_pred = model.predict(X_val)
round((y_val == y_pred).mean(), 2)

np.float64(0.9)

> A/ ```0.9```

# Question 5

* Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?
* age
* balance
* marital
* previous


In [236]:
dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(dicts)
model.fit(X_train, y_train)

dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.fit_transform(dicts)
y_pred = model.predict(X_val)
original_accuracy = (y_val == y_pred).mean()
print('Original:', original_accuracy)

for f in categorical + numerical:
    smaller_features = [feature for feature in categorical + numerical if feature != f]
    dicts_smaller = df_train[smaller_features].to_dict(orient='records')
    X_train = dv.fit_transform(dicts_smaller)
    model.fit(X_train, y_train)

    dicts = df_val[smaller_features].to_dict(orient='records')
    X_val = dv.fit_transform(dicts)
    y_pred = model.predict(X_val)
    print(f'Without {f} difference:', original_accuracy - (y_val == y_pred).mean())


Original: 0.900353904003539
Without job difference: -0.0011059500110595089
Without marital difference: 0.00022119000221187957
Without education difference: -0.00044238000442387015
Without housing difference: -0.0001105950011059953
Without contact difference: -0.00033178500331787486
Without month difference: 0.0005529750055296434
Without poutcome difference: 0.006525105065250947
Without age difference: -0.0008847600088476293
Without balance difference: -0.0006635700066357497
Without day difference: -0.00044238000442387015
Without duration difference: 0.010395930103959228
Without campaign difference: -0.0002211900022119906
Without pdays difference: -0.0012165450121655041
Without previous difference: -0.0006635700066357497


> A/ ```marital```

# Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?
* 0.01
* 0.1
* 1
* 10
* 100


In [220]:
for C in [0.01, 0.1, 1, 10, 100]:
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(dicts)
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.fit_transform(dicts)
    y_pred = model.predict(X_val)
    accuracy = round((y_val == y_pred).mean(), 3)
    print(f"{C}:", accuracy)

0.01: 0.898
0.1: 0.901
1: 0.901
10: 0.901
100: 0.9


> A/ ```0.1```